# Preprocessing

Rename Masbaisis_indices_280619.xlsx to Masbasis_indices_280619.xlsx  
Rename Masbasis_2606_2019_color_and_othe_indecies  to Masbasis_260619_color_and_othe_indecies 

In [1]:
%%time

import pandas as pd
import numpy as np
import math
import os
import datetime
from copy import copy
from pprint import pprint
import json

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# To display df nicely in loops
from IPython.display import display 
# display(df1.head()) 
# display(df2.head())

# Display rows and columns Pandas
pd.options.display.max_columns = 100
pd.set_option('display.max_rows',100)

Wall time: 1.18 s


In [2]:
# Prints the current working directory
os.getcwd()
# os.listdir()

'C:\\Users\\fahad\\MegaSync\\NMBU\\GitHub\\vPheno'

## Finding Username folder to make general path for multi PC use

In [3]:
username = str(os.getcwd()).split('\\')[2]
username

'fahad'

## ToDo:  
  
2019 2020 data is fine  
Cant use 2017 because of the blue band  
Distt is not normal in 2018 robot  
AREA UNDER THE CURVE in each season for 2018 2017  

## Doing Manual work to involve grain yield
## ToDo: Must automate

## Importing Data

In [4]:
path = r'C:\\Users\\'+username+'\\MegaSync\\NMBU\\Master Thesis\\Data\\Feb2021'
export_path = './Data/'
complete_df_path = export_path+'complete/'
incomplete_df_path = export_path+'incomplete/'
os.listdir(path)

['Graminor_2019', 'Masbasis_2019', 'Staur_2019', 'Vollebekke-total_2020']

## Data Preparation
### Creating list of all files

In [5]:
# Get the list of all files in directory tree at given path

files_with_address = []
files_list = []

for (dirpath, dirnames, filenames) in os.walk(path):
    files_with_address += [os.path.join(dirpath, file) for file in filenames]
    files_list.extend(filenames)
    
print(len(files_with_address), 'files founnd in the directory')
# files_with_address
# files_list
files_with_address_bkp = copy(files_with_address)

74 files founnd in the directory


## Data Checking/control

### Check for duplicate filenames

In [6]:
print('Total number of files are :', len(files_list))

print('Number of unique file names are:', len(set(files_list)))

print('There is/are', len(files_list) - len(set(files_list)),'duplicate file name/names.')

Total number of files are : 74
Number of unique file names are: 73
There is/are 1 duplicate file name/names.


### Checking if there are multiple sheets in the files

In [7]:
# Print number of sheets in all files
print('The following files have multiple sheets.')

list_multi_sheet = []
for file in files_with_address:
    
    xl_file = pd.ExcelFile(file,engine='openpyxl')
    number_of_sheets = len(xl_file.sheet_names)
    if number_of_sheets > 1:
        print(number_of_sheets, os.path.basename(file), 'in folder', os.path.basename(os.path.dirname(file))
)
        list_multi_sheet.append(file)

The following files have multiple sheets.
4 19TvPhenores.xlsx in folder Graminor_2019
4 19TvPhenores pedigree.xlsx in folder Staur_2019
4 19TvPhenores.xlsx in folder Staur_2019
3 20BMLGI1_2020_tm.xlsx in folder Masbasis
3 Masbasis_Mica_2020_all_dates_MEDIAN_DP.xlsx in folder Masbasis
3 ROBOT_2020.xlsx in folder Robot


### Removing files with multiple sheets from the list

In [8]:
# Removing files with multiple sheets from the list

for i in list_multi_sheet:
    files_with_address.remove(i)
len(files_with_address)

68

### Removing files without dates
(with 2019 in name means they dont have date format)

In [9]:
files_w_2019 = []
for file in files_with_address:
    file_name = os.path.basename(file)
    if '2019' in file_name:
        print(file_name, 'in folder', os.path.basename(os.path.dirname(file)))
        files_w_2019.append(file)

Field_data_2019.xlsx in folder Masbasis_2019
Masbasis_2019_ForDP_median.xlsx in folder Masbasis_2019
NEW_Field_data_2019.xlsx in folder Masbasis_2019
Staur_Graminor_2019_median.xlsx in folder Staur_2019
Staur_Masbasis_2019.xlsx in folder Staur_2019


In [10]:
# Removing from list
for i in files_w_2019:
    print(i)
    files_with_address.remove(i)

C:\\Users\\fahad\MegaSync\NMBU\Master Thesis\Data\Feb2021\Masbasis_2019\Field_data_2019.xlsx
C:\\Users\\fahad\MegaSync\NMBU\Master Thesis\Data\Feb2021\Masbasis_2019\Masbasis_2019_ForDP_median.xlsx
C:\\Users\\fahad\MegaSync\NMBU\Master Thesis\Data\Feb2021\Masbasis_2019\NEW_Field_data_2019.xlsx
C:\\Users\\fahad\MegaSync\NMBU\Master Thesis\Data\Feb2021\Staur_2019\Staur_Graminor_2019_median.xlsx
C:\\Users\\fahad\MegaSync\NMBU\Master Thesis\Data\Feb2021\Staur_2019\Staur_Masbasis_2019.xlsx


In [11]:
len(files_with_address)
# files_with_address

63

### Checking number of unique cultivars in the field

In [12]:
# plots_data = pd.read_excel(files_with_address[0],engine='openpyxl')
# # Pandas converts 'NA' string to NaN. Need to change those to 
# # some string to get a count as NaNs are not counted as unique values

# plots_data.Name.fillna('-', inplace=True)
# plots_data.CodeName.fillna('-', inplace=True)

# # Creating a new column as multiple plots were named 'NA' but the 
# # CodeName was different for each one of them
# plots_data['NameCode'] = plots_data.Name+plots_data.CodeName

# plots_data
# len(plots_data.NameCode.unique())
# plots_data.NameCode.value_counts()
# # plots_data.NameCode.value_counts().sum()
# # plots_data

## Importing data files to Pandas

In [13]:
%%time

all_df = []
for data in files_with_address:
    file_name = os.path.splitext(os.path.basename(data))[0]

    # Replce all invalid characters in the name
    file_name = file_name.replace("-", "_")
    file_name = file_name.replace(" ", "_")
    file_name = file_name.replace("(", "")
    file_name = file_name.replace(")", "")
    df_name = file_name.replace(".", "")
    # Test: Check if the same date is already present in the current dict key
    if df_name in all_df:
        print(f'A file with the same name {df_name} has already been imported. \n Please check if there is duplication of data.')
        raise NameError
            
    all_df.append(df_name)

    locals()[df_name] = pd.read_excel(data, engine='openpyxl')
    print(df_name, '=====', locals()[df_name].shape)

Graminor_070819_2 ===== (600, 49)
Graminor_150819 ===== (600, 49)
Graminor_eastwest_050819 ===== (600, 49)
Graminor_250719 ===== (600, 46)
Graminor_050719_one_missing_row_deleted ===== (600, 49)
Graminor_150719 ===== (600, 49)
Graminor_220719_corrected ===== (600, 49)
Graminor_eastwest_020719_NIR_half_missing ===== (600, 22)
Graminor_east_110719 ===== (300, 49)
Graminor_060619 ===== (600, 49)
Graminor_110619 ===== (600, 49)
Graminor_280619_corrected ===== (600, 49)
Graminor_eastwest_250619_correct_duplicate_east ===== (600, 97)
Masbasis_050719_corrected ===== (528, 49)
Masbasis_060619_Indices ===== (528, 56)
Masbasis_070819_correct ===== (528, 49)
Masbasis_110619 ===== (528, 82)
Masbasis_150719 ===== (528, 49)
Masbasis_220719_correct ===== (528, 49)
Masbasis_260619_color_and_othe_indecies ===== (528, 47)
Masbasis_290719 ===== (528, 49)
Masbasis_indices_280619 ===== (528, 49)
Graminor_eastwest_040720 ===== (800, 49)
Graminor_eastwest_040820 ===== (793, 22)
Graminor_eastwest_070720_corre

In [14]:
print(f'Total imported {len(all_df)}')

Total imported 63


## Data Control: Check which df have the data column heading we need

### Listing column headings into df

In [15]:
%%time

# Finding max number of columns in all df

len_columns = []
for df in all_df:
    cols_df = locals()[df].columns
    len_columns.append(len(cols_df))
max_cols_in_df = max(len_columns)

# Now creating a empty df to collect all column headings
columns_df = pd.DataFrame(data=range(0,max_cols_in_df), columns = ['ID'])
columns_df.drop('ID', axis=1, inplace=True)

for df in all_df:
    cols_df = locals()[df].columns
    columns_df[df] = pd.Series(cols_df)


Wall time: 50.9 ms


### Convert columns_df to dictoionary

In [16]:
%%time

# Function to convert df to dict while dropping nan in each column separately

def comp_dropna(df1):
    return {k: v.dropna().to_dict() for k,v in df1.items()}

columns_dict = comp_dropna(columns_df)
# columns_dict

Wall time: 12 ms


### Finding Column heading with the text we are looking for in

In [17]:
%%time

print(f'Total {len(all_df)} datasets have been imported.')

# Find columns with key from the following list
# Rename the founc columns to the values against the keys

id_cols_var = ['id', 'plot']
id_cols_new = ['Plot_ID']


find_cols = {
    id_cols_var[0]: id_cols_new[0],
    id_cols_var[1]: id_cols_new[0],
    'Blue': 'Blue',
    'Gre': 'Green',
    'Red': 'Red',
    'RedE': 'RedEdge',
    'NIR': 'NIR',
#     'NDVI': 'NDVI',
#     'MTCI': 'MTCI'
}


dtype_f = {0: '_Mean_Value', 1: '_Median_Value', 2: '_Std_Dev_Value'}
num_std_dtype_colunms = len(dtype_f)

list_ok_df_per_query = []
list_problem_df_per_query = []

for query, query_f in find_cols.items():
    
    # First, we loop through all df to find the max of number of relavant columns in any df
    num_col_found_all = []
    for df, cols_dict in columns_dict.items():

        # List to save the relavant columns found in the df
        cols_found = []
        
        # Loop through all column names
        for key, item in cols_dict.items():
            
            # Check if required text is in colmn name,i.e. item
            
            # Ensure first that the text does not contain 'count' and 'sum' in it
            if item.lower().find('sum'.lower()) == -1 and item.lower().find('count'.lower()) == -1:
            
                # When searching for Red, search for red but ignore RedEdge,i.e. RedE
                if query.lower()=='red':
                    if item.lower().find(query.lower()) != -1 and item.lower().find('RedE'.lower()) == -1:
                        cols_found.append(item)

                # For the rest also checking we do not get 'GNDVI' columns. Don't neet them yet
                elif item.lower().find(query.lower()) != -1 and item.lower().find('GNDVI'.lower()) == -1:
                    cols_found.append(item)

        num_col_found_all.append(len(cols_found))
    max_cols = max(num_col_found_all)
#     print(query, max_cols)

    # Define a df for saving the columns found if num of col is not num_std_dtype_colunms
    temp_df_non_std_cols = 'col_df_non_std_cols_'+query
    
    # Add a condition in data if no columns found, then still create a df with 1 column
    locals()[temp_df_non_std_cols] = pd.DataFrame(data=range(0,max_cols) if max_cols > 1 else range(0,1),
                                                  columns = [query])
    list_problem_df_per_query.append(temp_df_non_std_cols)
    
    # Define a df for saving the columns found if num of col is num_std_dtype_colunms
    temp_df_std_cols= 'col_df_std_cols_'+query
    locals()[temp_df_std_cols] = pd.DataFrame(data=range(0,num_std_dtype_colunms), columns = [query])
    list_ok_df_per_query.append(temp_df_std_cols)
    
    
    
    # Now we again loop and append those columns to a relavant df for each query
    
    for df, cols_dict in columns_dict.items():

        # List to save the relavant columns found in the df
        cols_found = []

        # Loop through all column names
        for key, item in cols_dict.items():

            # Check if required text is in colmn name,i.e. item
            
            # Ensure first that the text does not contain 'count' and 'sum' in it
            if item.lower().find('sum'.lower()) == -1 and item.lower().find('count'.lower()) == -1 and item.lower().find('GreenND'.lower()) == -1:
            
                # When searching for Red, search for red but ignore RedEdge,i.e. RedE
                if query.lower()=='red':
                    if item.lower().find(query.lower()) != -1 and item.lower().find('RedE'.lower()) == -1:
                        cols_found.append(item)

                # For the rest also checking we do not get 'GNDVI' columns. Don't neet them yet
                elif item.lower().find(query.lower()) != -1 and item.lower().find('GNDVI'.lower()) == -1:
                    cols_found.append(item)

        # Adding the found values to a column in dataframe
        
        # For non standard columns
        if len(cols_found) != num_std_dtype_colunms and query not in id_cols_var:
            locals()[temp_df_non_std_cols][df] = pd.Series(cols_found, dtype='str')
        
        # For standrd columns
        if len(cols_found) == num_std_dtype_colunms or query in id_cols_var:
            locals()[temp_df_std_cols][df] = pd.Series(cols_found, dtype='str')
            

        if query in id_cols_var:
            num_id_cols = 0
            for temp_query in id_cols_var:
                for column in cols_dict.values():
                    if column.lower().find(temp_query.lower()) == 0:
                        num_id_cols += 1
            if num_id_cols !=1:
                if num_id_cols == 0:
                    print(query, 'Error: ID column not found in' ,df)
                else:
                    print('Error: More than one ID column found in ' ,df)
    
    # Dropping the empty query named column from the columns dataframe
    # For non standard columns
    locals()[temp_df_non_std_cols].drop(query, axis=1, inplace=True)
    # For standrd columns
    locals()[temp_df_std_cols].drop(query, axis=1, inplace=True)

    
    # Drop extra empty rows if the query is id or plot
    # Drop empty columns, where the id is different than the one being checked
    # Drop first column with 
    # Assumption: There is only one id columns in each df
    # Assumption being checked in previous loop
    if query in id_cols_var:
        locals()[temp_df_std_cols].drop([1, 2], axis=0, inplace=True)
        locals()[temp_df_std_cols].dropna(axis=1, inplace=True)
    
    
#     # ===========================Start_of_Analysis_1================================
#     # Analysis Part: Uncomment for new data
#     # Confirm if all row elements corresopnd to data aggregation types in the dict
    
    
#     agg_types_dict = {0:'mea', 1:'med', 2:'st'}
    
#     if query in id_cols_var:
#         pass
#     else:
#         for key, agg_type in agg_types_dict.items():
#             check_df = locals()[temp_df_std_cols]#.drop(query, axis=1)
#             for row_item in check_df.iloc[key].tolist():
#                 if str(row_item).lower().find(agg_type) == -1:
#                     print(f'{row_item}, {key}, {agg_type}')
                
#     print(query, locals()[temp_df_std_cols].shape)
#     display(locals()[temp_df_std_cols])
    
#     # Analysis shows that for df with num_std_dtype_colunms relavant column names; 
#     # green and RedEdge columns have naming problems where the agg_types are
#     # not the same for mea and med for some green columns
#     # and mea, med, and st for most of the RedEdge columns
#     # =============================End_of_Analysis_1================================





    # ===========================Start_of_Analysis_2================================
    # Analysis Part: Uncomment for new data
    # Renaming the column names to standard names 
    # Renaming based on the rule that the first name is mean, second is median and 
    # the third is standard deviation.
    # This assumption has been partially checked and confirmed in Analysis_1
    
    # The id column has to be dealed separately since it does not have 
    # num_std_dtype_colunms variations i.e. mean, median, and std_dev
    df_found_correct_cols = locals()[temp_df_std_cols].shape[1]
    renamed_count = 0
    if query in id_cols_var:
        for x, y in locals()[temp_df_std_cols].items():
            locals()[x].rename({y[0]: query_f}, axis=1, inplace=True)
#             print(f'Renamed {query_f} column for dataframe {x} against value {y[0]}')
            renamed_count += 1

    else:
        for x, y in locals()[temp_df_std_cols].items():
            locals()[x].rename({y[0]: query_f+dtype_f[0], y[1]: query_f+dtype_f[1],\
                                y[2]: query_f+dtype_f[2]}, axis=1, inplace=True)
#             print(f'Renamed {query_f} columns for dataframe {x}')
            renamed_count += 1
    print(f'{query} => {query_f} column renamed in {renamed_count} of {df_found_correct_cols} found dataframes')
#     display(locals()[temp_df_std_cols].shape[2])
    
    # =============================End_of_Analysis_2================================

    
    
    
    
    
    
#     # ===========================Start_of_Analysis_3================================
#     # Analysis Part: Uncomment for new data
#     # Checking the uncommon column names/combination or df with extra columns that 
#     # make finding the correct data confusing

#     print(temp_df_non_std_cols)
#     display(locals()[temp_df_non_std_cols])

#     # =============================End_of_Analysis_3================================
    




Total 63 datasets have been imported.
id => Plot_ID column renamed in 54 of 54 found dataframes
plot => Plot_ID column renamed in 9 of 9 found dataframes
Blue => Blue column renamed in 58 of 58 found dataframes
Gre => Green column renamed in 58 of 58 found dataframes
Red => Red column renamed in 58 of 58 found dataframes
RedE => RedEdge column renamed in 58 of 58 found dataframes
NIR => NIR column renamed in 57 of 57 found dataframes
Wall time: 414 ms


### List of df with problems

In [18]:
%%time

list_problem_df_per_query
problem_df = []
for df in list_problem_df_per_query:
    problem_df.extend(locals()[df].columns)

print(len(set(problem_df)))
set(problem_df)

7
Wall time: 999 µs


{'Graminor_220719_corrected',
 'Graminor_eastwest_090720_duplication_same_as_010720',
 'Graminor_eastwest_250619_correct_duplicate_east',
 'Masbasis_110619',
 'Robot_Mica_070820',
 'Robot_Mica_140820',
 'Robot_Mica_160720'}

Probles after merging eastwest in Graminor  
 'Graminor_220719_corrected' NIR Missing  
 'Graminor_eastwest_090720_duplication-same_as_010720' duplication  
 'Graminor_eastwest_250619_correct_duplicate_east' duplication  
 'Masbasis_110619' duplication  
 'Robot_Mica_070820' duplication  
 'Robot_Mica_140820' duplication  
 'Robot_Mica_160720' NIR Missing  

Problem datasets before combining east west datasets in Graminor{'Graminor_220719_corrected',  
 'Graminor_east_090720'  
 'Graminor_east_250619_corrected' 
 'Graminor_west_020719'   
 'Masbasis_110619'   
 'Robot_Mica_070820'   
 'Robot_Mica_140820'   
 'Robot_Mica_160720'
 

Dataframes with Problems (begining)

 'Graminor_070819_2'  
 'Graminor_220719_corrected'  
 'Graminor_east_010720'  
 'Graminor_east_090720'  
 'Graminor_east_250619_corrected'  
 'Graminor_west_020719'  
 'Masbasis_Mica_130720'  
 'Robot_Mica_070820'  
 'Robot_Mica_140820'  
 'Robot_Mica_160720'  

## Standardise the column names
Get list of df with all required columns after implementing standard names 

### ToDo: Test: Check if there is duplication of column names

### List down standard column names

In [19]:
%%time

std_columns = []
std_columns_mean = []
std_columns_median = []
std_columns_stdev = []

for key, value in find_cols.items():
    if key in id_cols_var:
        std_columns.append(value)
        std_columns_mean.append(value)
        std_columns_median.append(value)
        std_columns_stdev.append(value)

    else:
        std_columns.extend([value+y for x, y in dtype_f.items()])
        std_columns_mean.append(value+dtype_f[0])
        std_columns_median.append(value+dtype_f[1])
        std_columns_stdev.append(value+dtype_f[2])
        
std_columns.remove('Plot_ID')
std_columns_mean.remove('Plot_ID')
std_columns_median.remove('Plot_ID')
std_columns_stdev.remove('Plot_ID')

std_columns
# std_columns_mean
# std_columns_median
# std_columns_stdev

Wall time: 0 ns


['Plot_ID',
 'Blue_Mean_Value',
 'Blue_Median_Value',
 'Blue_Std_Dev_Value',
 'Green_Mean_Value',
 'Green_Median_Value',
 'Green_Std_Dev_Value',
 'Red_Mean_Value',
 'Red_Median_Value',
 'Red_Std_Dev_Value',
 'RedEdge_Mean_Value',
 'RedEdge_Median_Value',
 'RedEdge_Std_Dev_Value',
 'NIR_Mean_Value',
 'NIR_Median_Value',
 'NIR_Std_Dev_Value']

In [20]:
find_cols

{'id': 'Plot_ID',
 'plot': 'Plot_ID',
 'Blue': 'Blue',
 'Gre': 'Green',
 'Red': 'Red',
 'RedE': 'RedEdge',
 'NIR': 'NIR'}

### Find df which have all the required std columns

In [21]:
%%time

complete_dataframes = []
incomplete_df = []

for df in all_df:
    df_columns = locals()[df].columns
    if (all(x in df_columns for x in std_columns)):
        complete_dataframes.append(df)
    else:
        incomplete_df.append(df)
print(f'Found {len(complete_dataframes)} complete dataframes. Can be accessed using \"complete_dataframes\" list')
print(f'Found {len(incomplete_df)} incomplete dataframes. Can be accessed using \"incomplete_df\" list')

Found 56 complete dataframes. Can be accessed using "complete_dataframes" list
Found 7 incomplete dataframes. Can be accessed using "incomplete_df" list
Wall time: 997 µs


## Exporting complete and incomplete datasets

In [22]:
# Exporting complete df
# Defining path
os.makedirs(complete_df_path, exist_ok=True)

# Test: Check if there are duplicate names in the complete_dataframes list
# As the last file with the same name will overwrite the previously exported file, resulting in data loss
if len(complete_dataframes) != len(set(complete_dataframes)):
    print('There are duplicate names in the complete_dataframes list.\nPlease check before exporting as it might result in data loss.')
        
    # Printing the names of the duplicate datasets, if any
    find_duplicates=[]
    for i in complete_dataframes:
        if i not in find_duplicates:
            find_duplicates.append(i)
        else:
            print(f'Duplicate dataset named \'{i}\'')
    raise NameError
    
for df in complete_dataframes:
    locals()[df].to_csv(complete_df_path+df+'.csv', index=False)

In [23]:
# Defining path
os.makedirs(incomplete_df_path, exist_ok=True)

# Test: Check if there are duplicate names in the incomplete_df list
# As the last file with the same name will overwrite the previously exported file, resulting in data loss
if len(incomplete_df) != len(set(incomplete_df)):
    print('There are duplicate names in the incomplete_df list.\nPlease check before exporting as it might result in data loss.')
    
    # Printing the names of the duplicate datasets
    find_duplicates=[]
    for i in incomplete_df:
        if i not in find_duplicates:
            find_duplicates.append(i)
        else:
            print(f'Duplicate dataset named \'{i}\'')
    raise NameError
    
for df in incomplete_df:
    locals()[df].to_csv(incomplete_df_path+df+'.csv', index=False)

In [24]:
import json
a_file = open("Data\std_columns.json", "w")
json.dump(std_columns, a_file)
a_file.close()

# a_file = open("Data\std_columns.json", "r")
# output = a_file.read()
# a_file.close()
# print(output)

# END OF SECTION